In [ ]:
import pandas as pd

df = pd.read_csv(r"C:\Users\zeyne\OneDrive\Desktop\hourly_transportation_202407.csv")

df.head()

,transition_date,transition_hour,transport_type_id,road_type,line,transfer_type,number_of_passage,number_of_passenger,product_kind,transaction_type_desc,town,line_name,station_poi_desc_cd
0,2024-07-01,0,1,OTOYOL,KADIKOY-RASATHANE-ANADOLU HISARI,Normal,6,6,TAM,Tam Abonman,BAKIRKOY,14R,NaN
1,2024-07-01,0,3,DENİZ,ESKIHISAR-TOPCULAR,Normal,1,1,TAM,Tam Kontur,CIFTLIKKOY,İDO-KRFZ,TOPCULAR
2,2024-07-01,0,3,DENİZ,ADALAR-BOSTANCI,Normal,3,2,NaN,Tam Kontur,ADALAR,ADALAR-BST,BURGAZADA
3,2024-07-01,0,2,RAYLI,USKUDAR-CEKMEKOY,Aktarma,1,1,INDIRIMLI2,Indirimli Tip 2 Aktarma,USKUDAR,M5,ALTUNIZADE 1
4,2024-07-01,0,2,RAYLI,TOPKAPI-HABIBLER,Normal,3,3,UCRETSIZ,Ucretsiz,SULTANGAZI,T4,YENIMAHALLE


In [ ]:
Veri seti CSV dosyasından okunur.
Analiz ve modelleme için gerekli temel sütunlar seçilir.
Sütun adları daha anlaşılır ve standart bir formata dönüştürülür.
Tarih, saat, yolcu sayısı gibi alanların veri tipleri doğru tipe çevrilir.
Metin içeren sütunlar temizlenip (trim, upper/lower) tutarlı hale getirilir.
Bu işlemler sonucunda veri seti temiz, düzenli ve analiz için uygun duruma getirilir

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\zeyne\Downloads\hourly_transportation_202408.csv")

df = df[['transition_date', 'transition_hour', 'line', 'road_type',
         'number_of_passenger', 'transport_type_id', 'town']]

df = df.rename(columns={
    'transition_date': 'date',
    'transition_hour': 'hour',
    'line': 'route_code',
    'road_type': 'stop_code',  # durak kodu yerine geçiyor
    'number_of_passenger': 'passenger_count',
    'transport_type_id': 'vehicle_type',
    'town': 'district'
})

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['hour'] = pd.to_numeric(df['hour'], errors='coerce').astype('Int64')

df['route_code'] = df['route_code'].astype(str).str.strip().str.upper()
df['stop_code'] = df['stop_code'].astype(str).str.strip().str.upper()
df['vehicle_type'] = df['vehicle_type'].astype(str).str.strip().str.lower()
df['passenger_count'] = pd.to_numeric(df['passenger_count'], errors='coerce')

print("Sütunlar dönüştürüldü ✅")
display(df.head(5))

Sütunlar dönüştürüldü ✅


,date,hour,route_code,stop_code,passenger_count,vehicle_type,district
0,2024-08-01,0,MESCIDI SELAM-ARNAVUTKOY-ISTANBUL HAVA LIMANI,OTOYOL,1,1,ATASEHIR
1,2024-08-01,0,MECIDIYEKOY-ISTANBUL HAVALIMANI,OTOYOL,1,1,ATASEHIR
2,2024-08-01,0,KOC UNV. RUMELIFENER KAMPUSU - TAKSIM,OTOYOL,3,1,ATASEHIR
3,2024-08-01,0,HACI OSMAN METRO - RUMELI FENERI,OTOYOL,1,1,ATASEHIR
4,2024-08-01,0,USKUDAR-GUZELTEPE-UMRANIYE DEVLET HASTANESI,OTOYOL,3,1,BAKIRKOY


DataFrame'deki toplam satır sayısını saklıyoruz.
"before" → temizleme işleminden önceki satır sayısı
Aynı tarih, saat, hat kodu, yol tipi ve araç türüne sahip tekrar eden satırlar kaldırılır.
Temizleme işleminden sonraki yeni satır sayısı.
Kaç tane yinelenen (duplicate) kaydın silindiğini ekrana yazdırıyoruz.

In [3]:
before = len(df)
df = df.drop_duplicates(subset=['date','hour','route_code','stop_code','vehicle_type'])
after = len(df)
print(f"Yinelenen kayıt sayısı: {before - after}")

Yinelenen kayıt sayısı: 2875543


DataFrame'deki tüm sütunlarda kaç adet eksik (NaN) değer olduğunu ekrana yazdırır.
Her bir route_code (hat) ve hour (saat) kombinasyonu için,
passenger_count içindeki eksik değerler o grubun "median" (ortanca) değeri ile doldurulur.
transform → aynı boyutta bir sonuç döndürerek gruplu doldurmayı sağlar.
Eğer hala eksik kalan varsa (örneğin medyan bulunamayan durumlarda),
tüm passenger_count sütunu için genel median ile dolduruyoruz.
passenger_count içerisinde eksik değer kalıp kalmadığını kontrol ediyoruz.

In [ ]:

print(df.isna().sum())

df['passenger_count'] = df.groupby(['route_code','hour'])['passenger_count'].transform(
    lambda x: x.fillna(x.median())
)
df['passenger_count'] = df['passenger_count'].fillna(df['passenger_count'].median())

print("\nEksik değer kalmadı mı?:", df['passenger_count'].isna().sum() == 0)

date                 0
hour                 0
route_code           0
stop_code            0
passenger_count      0
vehicle_type         0
district           136
dtype: int64

Eksik değer kalmadı mı?: True


DataFrame içindeki tüm sütun adlarını bir liste halinde döndürür.

In [5]:
df.columns.tolist()

['date',
 'hour',
 'route_code',
 'stop_code',
 'passenger_count',
 'vehicle_type',
 'district']

Negatif yolcu sayılarını siler
Saat 0-23 aralığında olmayanları siler

In [ ]:

df = df[(df['passenger_count'] >= 0)]

df = df[df['hour'].between(0, 23)]

'date' sütununu datetime formatına dönüştürüyoruz.
Veri setinindeki aylara göre filtreliyoruz.

In [7]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

df = df[(df['date'] >= "2024-08-01") & (df['date'] <= "2024-08-31")]

Kategorik veri tipine dönüştürülmesini istediğimiz sütunları liste halinde tanımlıyoruz.
Bu sütunlar: hat kodu, yol tipi, araç türü ve ilçe bilgisidir.
Listedeki her sütun için döngü başlatıyoruz
Her kolonu 'category' veri tipine dönüştürüyoruz.

In [8]:
cat_cols = ['route_code', 'stop_code', 'vehicle_type', 'district']

for col in cat_cols:
    df[col] = df[col].astype("category")

cat_cols içindeki kategorik sütunların tamamını ele alıyoruz ve apply ile hepsine işlem uyguluyoruz.

In [9]:
df[cat_cols] = df[cat_cols].apply(lambda x: x.str.strip() if x.dtype == "object" else x)


Tüm sütunları göz önünde bulundurarak veri setindeki tekrar eden satırları siler.
Temizleme işleminden sonra DataFrame'in yeni boyutunu (satır, sütun) ekrana yazdırır.

In [10]:
df = df.drop_duplicates()
print("Yeni boyut:", df.shape)

Yeni boyut: (78563, 7)


In [11]:
import matplotlib.pyplot as plt
import seaborn as sns

 Yolcu sayısı sütununun alt çeyrek (Q1) değerini hesaplar.
 Yolcu sayısı sütununun üst çeyrek (Q3) değerini hesaplar.
 IQR (Interquartile Range) yani çeyrekler arası genişlik.
 passenger_count değeri alt/üst sınırların dışındaysa AYKIRIDIR.
 Kaç tane aykırı (1) ve normal (0) değer olduğunu sayar.

In [17]:
Q1 = df['passenger_count'].quantile(0.25)
Q3 = df['passenger_count'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df['is_outlier'] = ((df['passenger_count'] < lower) | (df['passenger_count'] > upper)).astype(int)
df['is_outlier'].value_counts()

is_outlier
0    67464
1    11099
Name: count, dtype: int64

'hour' sütunundaki her bir değere bir fonksiyon (lambda) uyguluyoruz.
 Saat 07:00–10:00 arası sabah yoğunluğu
 Saat 16:00–20:00 arası akşam yoğunluğu
 Bu aralıklardaysa 1, değilse 0 döndürür.

In [19]:
df['is_peak_hour'] = df['hour'].apply(
    lambda x: 1 if (7 <= x <= 10) or (16 <= x <= 20) else 0
)


'date' sütunundan haftanın gününü çıkarıyoruz.
 dayofweek: Pazartesi = 0, Pazar = 6 olacak şekilde numaralandırır.
 day_of_week değerine bakarak hafta sonunu işaretliyoruz.

In [20]:
df['day_of_week'] = df['date'].dt.dayofweek  # 0=Mon, 6=Sun
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)


route_code (hat kodu) ve hour (saat) bilgisine göre gruplama yapıyoruz.
passenger_count sütununun grup içindeki ortalamasını hesaplıyoruz.


In [21]:
df['mean_passenger_by_route_hour'] = (
    df.groupby(['route_code', 'hour'])['passenger_count']
      .transform('mean')
)


C:\Users\zeyne\AppData\Local\Temp\ipykernel_20016\3190882940.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['route_code', 'hour'])['passenger_count']


DataFrame'i CSV formatında belirtilen dosya adıyla dışa aktarıyoruz.
index=False → satır numaralarının (index) dosyaya yazılmasını engeller.
Dosyanın başarıyla kaydedildiğini kullanıcıya bildiriyoruz.

In [22]:
df.to_csv("istanbul_temmuz_clean_eda.csv", index=False)
print("Kaydedildi.")

Kaydedildi.
